In [8]:
%reload_ext autoreload
%autoreload 2

import niseko

context = niseko.get_context("/Users/zeyuan.shang/Coding/niseko/data")

In [9]:
import time

datasets = list(context.list_datasets())
processed_num_datasets = 0
total_num_datasets = len(datasets)
one_hot_encoding = {
    'CLASSIFICATION': {},
    'REGRESSION': {}
}
scaling = {
    'CLASSIFICATION': {},
    'REGRESSION': {}
}
scaling_model = {}
feature_selection = {
    'CLASSIFICATION': {},
    'REGRESSION': {}
}
start = time.time()
one_hot_encoder_bad_datasets = []
encoder_applicable_num_datasets = 0
for ds in context.list_datasets():
    one_hot_encoder_votes = 0
    for pipeline in ds.get_pipelines(order_by='performance', num=10):
        if 'LabelEncoder' in pipeline.primitives:
            if 'OneHotEncoder' in pipeline.primitives:
                one_hot_encoding[ds.task_type]['with'] = one_hot_encoding[ds.task_type].get('with', 0) + 1
                one_hot_encoder_votes += 1
            else:
                one_hot_encoding[ds.task_type]['without'] = one_hot_encoding[ds.task_type].get('without', 0) + 1
        for scaler in ['MinMaxScaler', 'StandardScaler', 'RobustScaler']:
            if scaler in pipeline.primitives:
                scaling[ds.task_type][scaler] = scaling[ds.task_type].get(scaler, 0) + 1

        for feature_selector in ['VarianceThreshold', 'SelectPercentile']:
            if feature_selector in pipeline.primitives:
                feature_selection[ds.task_type]['with'] = feature_selection[ds.task_type].get('with', 0) + 1
            else:
                feature_selection[ds.task_type]['without'] = feature_selection[ds.task_type].get('without', 0) + 1
    
    ds_scaling_model = {}
    for pipeline in ds.get_pipelines(order_by='performance'):
        if pipeline.model not in scaling_model:
            scaling_model[pipeline.model] = {}
        has_scaler = False
        for scaler in ['MinMaxScaler', 'StandardScaler', 'RobustScaler']:
            if scaler in pipeline.primitives:
                has_scaler = True
        if not has_scaler:
            continue
        ds_scaling_model[pipeline.model] = ds_scaling_model.get(pipeline.model, 0) + 1
        if ds_scaling_model[pipeline.model] > 10:
            continue
        for scaler in ['MinMaxScaler', 'StandardScaler', 'RobustScaler']:
            if scaler in pipeline.primitives:
                scaling_model[pipeline.model][scaler] = scaling_model[pipeline.model].get(scaler, 0) + 1
                
    
    if 'LabelEncoder' in pipeline.primitives:
        encoder_applicable_num_datasets += 1
        if one_hot_encoder_votes < 5:
            one_hot_encoder_bad_datasets.append(ds.dataset_id)
    processed_num_datasets += 1
    print('{}/{}'.format(processed_num_datasets, total_num_datasets), time.time() - start)

print(one_hot_encoding)
print(scaling)
print(feature_selection)
print(scaling_model)

1/300 0.6446859836578369
2/300 0.6532988548278809
3/300 0.7950019836425781
4/300 0.8737950325012207
5/300 1.0447371006011963
6/300 1.07277512550354
7/300 1.2141308784484863
8/300 1.395738124847412
9/300 1.517937183380127
10/300 1.5671920776367188
11/300 1.5982089042663574
12/300 1.7185111045837402
13/300 2.1143531799316406
14/300 2.2015950679779053
15/300 2.2361299991607666
16/300 2.2869210243225098
17/300 2.306502103805542
18/300 2.3675551414489746
19/300 2.3799548149108887
20/300 2.4174540042877197
21/300 2.438446044921875
22/300 2.454164981842041
23/300 2.5187339782714844
24/300 2.5867152214050293
25/300 2.670517921447754
26/300 2.885793924331665
27/300 3.002903938293457
28/300 3.3754029273986816
29/300 3.7239999771118164
30/300 4.198759078979492
31/300 4.469206094741821
32/300 4.787629127502441
33/300 5.04000997543335
34/300 5.336846113204956
35/300 5.603752851486206
36/300 5.96763014793396
37/300 5.993263006210327
38/300 6.016417026519775
39/300 6.249261140823364
40/300 6.27262687

In [12]:
print(encoder_applicable_num_datasets)

print(one_hot_encoder_bad_datasets)

74
['299_libras_move', '313_spectrometer', '38_sick', 'LL0_1026_grub_damage', 'LL0_1245_lungcancer_shedden', 'LL0_13_breast_cancer', 'LL0_1551_autouniv_au6_400', 'LL0_1553_autoUniv_au7_700', 'LL0_179_adult', 'LL0_21_car', 'LL0_285_flags', 'LL0_454_analcatdata_halloffame', 'LL0_50_tic_tac_toe', 'LL0_555_analcatdata_apnea3', 'LL0_557_analcatdata_apnea1', 'LL0_567_kdd_coil_1', 'LL0_56_vote', 'LL0_6332_cylinder_bands', 'LL0_747_servo', 'LL0_840_autoHorse', 'LL0_968_analcatdata_birthday', 'LL0_uci_facebook_metrics', 'LL0_uci_las_vegas_strip', 'uu3_world_development_indicators']
